# calculating video frames count of all videos

In [ ]:
import json
import glob
import numpy as np
import cv2
import copy
import os
#change the path accordingly
video_files =  os.listdir('fake_videos')

print(len(video_files))
frame_count = []
sum=0
count=0
for video_file in video_files:
    f='fake_videos'+"/"+video_file
    cap = cv2.VideoCapture(f)
    frames_len=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count.append(frames_len)
    sum=sum+frames_len
    #if frames_len <170:
       # count+=1
print("frames" , frame_count)
print (sum)
#print (count)

#print("Total number of videos: " , len(frame_count))
#print('Average frame per video:',np.mean(frame_count))

# saving frames of videos in .jpg format of fake and real videos

In [ ]:
import cv2
import os
classifier = cv2.CascadeClassifier('hcd.xml') 
# Opens the Video file
directory=os.listdir('fake_videos')
for video_file in directory:
    f='fake_videos'+"/"+video_file
    cap= cv2.VideoCapture(f)
    i=0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
           
        if i%50==0:
            faces = classifier.detectMultiScale(frame,minNeighbors=30) 
            for (x, y, w, h) in faces:
                roi = frame[y:y+h, x:x+w]
                #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.imwrite(video_file +str(i)+'.jpg',roi)
        #cv2.imshow('live',frame)
        cv2.waitKey(1)
        i+=1    
    cap.release()
    cv2.destroyAllWindows()

#  preprocessing data 

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization
import matplotlib.pyplot as plt
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.0


In [2]:
idg=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, # You can uncomment these parameters to make you generator rotate & flip the images to put the train model in stricter conditions.
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    #validation_split=0.2
)

In [ ]:
train_gen = idg.flow_from_directory("data2/train_data",
                                                   target_size=(128,128),                                                   
                                                   subset='training',
                                                   class_mode='binary',
                                                   batch_size=32,
                                                   shuffle=True,
                                                   seed=1
                                                )

In [ ]:
train_gen.class_indices

In [ ]:
print(train_gen)

In [ ]:
val_gen = idg.flow_from_directory("data2/validation",
                                                   target_size=(128,128),                                                   
                                                   #subset='validation',
                                                   class_mode='binary',
                                                   batch_size=32,
                                                   shuffle=True,
                                                 
                                                )

# importing some pretrained models

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101

In [ ]:

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
input_shape = (128,128, 3)
googleNet_model =ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
googleNet_model.trainable = True

model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())

model.add(Dense(units=2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss',
                              # min_delta=0,
                              # patience=3,
                              # verbose=0,
                              # restore_best_weights = True)

checkpoint=tf.keras.callbacks.ModelCheckpoint(
   'final_InceptionV3.h5',
    save_best_only=True)




EPOCHS = 20
BATCH_SIZE =32
history = model.fit(train_gen, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = val_gen, verbose = 1,callbacks=checkpoint)

In [ ]:
model.save('deepfake_detection.h5')

In [ ]:
from keras.models import load_model

model=load_model("deepfake_detection.h5")

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data=[]
path='test'
for vid in os.listdir(path):
    if vid!='test1':
        f=path+"/"+vid 
        data.append(vid)  
    

In [ ]:
len(data)

In [ ]:
df=pd.DataFrame(data,columns = ['id'])

In [ ]:
Label='fake'

df['label']=Label
df['real']=0
df['fake']=0

In [ ]:
df

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))
t = f.suptitle('Pre-trained Performance ', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,EPOCHS+1))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, EPOCHS+1, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch #')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, EPOCHS+1, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch #')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

# now for test videos predicting for every frame

In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
path='test'
i=0
for vid in os.listdir(path):
    if vid!='test1':
        f=path+"/"+vid 
        cap = cv2.VideoCapture(f)

        classifier = cv2.CascadeClassifier('hcd.xml')
        real=0
        fake=0
        j=0

        frame_count = []
        frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
        #print(cap)   
         #print(cap)   
        while (cap.isOpened()):
            (ret, im) = cap.read()
            if ret == True:
                if j%8==0:
                    face = classifier.detectMultiScale(im,minNeighbors=20)
                    for (x, y, w, h) in face:
                        roi = im[y:y+h, x:x+w]
                        resized=cv2.resize(roi,(128,128))
                        resized_arr = image.img_to_array(resized)
                        resized_arr=resized_arr/255
                        resized_arr = np.expand_dims(resized_arr, axis = 0)
                        result = model.predict(resized_arr)
                        #print(result)
                        predicted_class_indices=np.argmax(result,axis=1)
                        #print(predicted_class_indices)
                        if predicted_class_indices==1:
                            real+=1
                        else:
                            fake+=1
                        cv2.waitKey(1)
                j+=1
            else:
                break
        cap.release()
        cv2.destroyAllWindows()
        #real_per=real/frame_count[0]
        #fake_per=fake/frame_count[0]
        #new_real=0.7*real
        if fake>real:
            print(i,vid,'fake',real,fake )
            df['label'][i]='fake'
            #df['fake'][i]=fake
            #df['real'][i]=real
        else:
            print(i,vid,'real',real,fake )

           # print(fake)
           # print(frame_count)
            df['label'][i]='real'
            #df['real'][i]=real
            #df['fake'][i]=fake
        i+=1    
    
df['label'].replace({'fake':0, 'real':1}, inplace=True)    
df.to_csv('deepfake_submission.csv',index=False)    